# Testing the created framework

In [ ]:
from utils import *
from pytorchNet import MLP, train_pytorch_model, generate_images, generate_contours, compute_nb_errors_torch
import framework as frw
import torch
import seaborn as sns
import glob
import cv2
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load dataset

In [ ]:
train_set, train_target = generate_disc_set(2500)
test_set, test_target = generate_disc_set(2500)

## Create a model & test forward

In [ ]:
model = create_model()
output = model.forward(train_set,no_grad=True)

## Test the backward and no_grad

### No_grad

In [ ]:
criterion = frw.MSELoss()
output = model.forward(train_set,no_grad=True)
loss = criterion.forward(output,train_target)
grdwrtoutput = criterion.backward()

In [ ]:
model.backward(grdwrtoutput)

### backward with grad

In [ ]:
output = model.forward(train_set,no_grad=False)
loss = criterion.forward(output,train_target)
grdwrtoutput = criterion.backward()

In [ ]:
model.backward(grdwrtoutput)

### Check the shapes of the gradients 

In [ ]:
for module in model.sequence:
    print(module)
    if module.params[module][0] is not None:
        print(module.params[module][0].shape)
        print(module.params[module][1].shape)

In [ ]:
print(model.params)

# Try to train a model 

In [ ]:
model = create_model()
params_frw = model.params
model_torch = MLP()
count = 0
with torch.no_grad():
    for par in model_torch.parameters():
        if params_frw[model.sequence[count]][0] is None:
            count += 1
        key = model.sequence[count]
        if len(par.shape) > 1:
            par -= par - params_frw[key][0][:,:-1]
        else:
            par -= par - params_frw[key][0][:,-1]
            count += 1

In [ ]:
nb_runs = 10

accuracy_test = accuracy_test_torch = 0
for k in range(nb_runs):
    model = create_model()
    model_torch = MLP()
    train_model(model,train_set,train_target,epochs=50)
    train_pytorch_model(model_torch,train_set,train_target,epochs=150)
    accuracy_test += compute_nb_errors(model,test_set,test_target)
    accuracy_test_torch += compute_nb_errors_torch(model_torch,train_set,train_target)
accuracy_test, accuracy_test_torch = accuracy_test/nb_runs, accuracy_test_torch/nb_runs

In [ ]:
print("accuracy on test set with fwk = ",round(accuracy_test,1))
print("accuracy on test set with pytorch = ",round(accuracy_test_torch,1))

# Visualization

In [ ]:
X = torch.linspace(0,1,1000)
Y = torch.linspace(0,1,1000)
grid_x, grid_y = torch.meshgrid(X,Y)
grid_x_vector = grid_x.reshape(-1,1)
grid_y_vector = grid_y.reshape(-1,1)
inputs = torch.cat((grid_x_vector,grid_y_vector),dim=1)
predicted = model.forward(inputs,no_grad=True)
predicted = predicted.reshape(grid_x.shape[0],-1)
with torch.no_grad():
    predicted_torch = model_torch(inputs)
    predicted_torch = predicted_torch.reshape(grid_x.shape[0],-1)

In [ ]:
fig = plt.figure(figsize=[16,7])
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
cs1 = ax1.contourf(grid_x,grid_y,predicted)
cs2 = ax2.contourf(grid_x,grid_y,predicted_torch)
ax1.set_title("Our framework")
ax2.set_title("Pytorch")
plt.show()

In [ ]:
data = assess_model(create_model,1,1,2)

In [ ]:
columns = ["Run","Accuracy train","Accuracy test","Epochs"]
data_pd = pd.DataFrame(data,columns=columns)
fig = plt.figure(figsize=[10,6])
ax = fig.add_subplot(1,1,1)
sns.set_style("darkgrid")
sns.lineplot(data=data_pd,x="Epochs",y="Accuracy train",ax=ax)
sns.lineplot(data=data_pd,x="Epochs",y="Accuracy test",ax=ax)
plt.show()

# GIF

In [ ]:
model = create_model()
model_torch = MLP()
steps=3
epochs=90
params_frw = model.params
count = 0
with torch.no_grad():
    for par in model_torch.parameters():
        if params_frw[model.sequence[count]][0] is None:
            count += 1
        key = model.sequence[count]
        if len(par.shape) > 1:
            par -= par - params_frw[key][0][:,:-1]
        else:
            par -= par - params_frw[key][0][:,-1]
            count += 1
generate_images(train_set,train_target,model,model_torch,steps,epochs,folder="figures")

In [ ]:
path_img = ["figures/epochs" + str(nb_epochs) + ".jpg" for nb_epochs in range(steps,epochs+1,steps)]
img_arr = []
for filename in path_img:
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_arr.append(img)
 

out = cv2.VideoWriter('mygif.avi',cv2.VideoWriter_fourcc(*'DIVX'), 3, size)
 
for i in range(len(img_arr)):
    out.write(img_arr[i])
out.release()